#### First steps with Value At Risk 

#### Definition of quantile for a real random variable (no Finance here)
For a real random variable $X$ we define the notion of quantile.

Even if not strictly necessary, we assume the variable to have
and strictly monotonic increasing partition function, so that
we can invert and take the quantile

#### We run here some simple simulations to compute quantiles

#### Definition of Value At Risk as $VaR_{\alpha}(V) = - Q_{1-c}(V)$

#### Interpreting the defintion and providing a coding example

#### Important remark: V is now still a generic random var.
In the next chapter, $V$ represents Financial stuff.